[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# Covid-19 Dashboard

This interactive dashboard displays the total number of males and females who died within 28 days of being identified as a COVID-19 case by a positive test since the start of the pandemic. It is further split up by age groups.

In [1]:
# import libraries

from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
import seaborn as sns

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable

# load data
with open("deaths.json", "rt") as INPUT:
    rawdata=json.load(INPUT)
    

In [4]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    # extract value of key 'data' which is a list of a dictionary
    datadictionary = rawdata['data'][0]
    datadictionary
    
    # extract dictionary values of 'MaleDeaths' and 'FemaleDeaths' keys' 
    # and assign to their own variables
    males = datadictionary['MaleDeaths']
    females = datadictionary['FemaleDeaths']

    # extract age bands - same for both lists

    # create empty list
    age_band = []

    # for each row in the list, extract the value of key 'age' and add to the list 'age_band'
    for i in males:
        age_band.append(i['age'])
    
    age_band.sort(key=min_age)

    # format list
    # create a new list to append
    age_groups = []

    # for each item in the age_band list
    for i in age_band:
        # replace the + and _ characters and append the new list
        age_groups.append(i.replace("_"," "))
    
    # Define dataframe
    deaths_df=pd.DataFrame(index=age_groups, columns=['males','females', 'total'])
    
    # loop over males and females values to fill in deaths_df (dataframe)
 
    # create counters
    num = 0
    num2 = 0

    # for each item in dictionary 'males'
    for i in males: 
        
        # ages is equal to age_groups[index] 
        ages = age_groups[num]
    
        # access specific location in frame
        deaths_df.loc[ages, 'males'] = i['value']
    
        num += 1
    
    for i in females:
        ages = age_groups[num2]
    
        deaths_df.loc[ages, 'females'] = i['value']
    
        num2 += 1
        
    # get the total amoount of deaths
    deaths_df['total'] = deaths_df['males'] + deaths_df['females']
    
    return deaths_df

# create a function to be used a sort key
def min_age(agerange):
    
    # apply string methods
    agerange=agerange.replace('+','') # remove the + from 90+
    start=agerange.split('_')[0] # split() returns a list with the '_' removed
    
    # typecast into an integer and return value
    return int(start)


# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. Call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(rawdata) # df is the dataframe for plotting

In [5]:
# our API access function. This will be called by the button when it is clicked
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    # we put the code for polling the api here
    
    # define filters
    filters = [
    'areaType=nation', # note each metric-value pair is inside one string
    'areaName=england'
    ]
    
    # values here are the names of the PHE metrics
    structure = {
    "MaleDeaths": "maleDeaths28Days",
    "FemaleDeaths": "femaleDeaths28Days"   
    }
    
    # create an instance of the object
    api = Cov19API(filters=filters, structure=structure)
    
    # calling the get_json() method of the api object 
    # sends the request to the API and retrieves the response
    deaths=api.get_json()
    
    # return data read from the API
    return deaths

Click the 'Refresh data' button to download the latest statistics:

In [6]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    
    # Get fresh data from the API.
    apidata=access_api()
    
    # we are calling a global variable df, important not to change this!
    # wrangle the data and overwrite the dataframe for plotting
    global df
    
    df=wrangle_data(apidata)
    
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction
    refresh_graph()
    
    # switch button to 'checked'
    apibutton.icon="user-check"
    
    # disable button
    apibutton.disabled=True


# create button widget    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # options are:'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='users' # (FontAwesome names without the `fa-` prefix)
)

# register button callback function with the button
apibutton.on_click(api_button_callback)

display(apibutton)


Button(description='Refresh data', icon='users', style=ButtonStyle(), tooltip='Click to download current Publi…

In [7]:
agecols=wdg.SelectMultiple(
    options=['males', 'females', 'total'], # options available
    value=['males', 'females'], # initial value
    rows=3, # rows of the selection box
    description='Gender',
    disabled=False
)

def age_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        
        # set default table style from seaborn library
        sns.set_style("darkgrid")
        
        # graphcolumns is a tuple - we need a list
        df.plot(kind='bar', y=list(graphcolumns)) 

        # add title and labels to chart
        plt.title("Deaths from COVID-19 within 28 days of a positive test")
        plt.xlabel("Age Group")
        plt.ylabel("No. of deaths")
        
        plt.show()
        
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
# this refreshes the graph when called  
def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. """
    
    current=agecols.value
    if current==(agecols.options[0], ):
        other=(agecols.options[1], )
    else:
        other=(agecols.options[0],)
    agecols.value=other # forces the redraw
    agecols.value=current # now we can change it back
    
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(age_graph, {'graphcolumns': agecols})

# display output
display(agecols, output)

SelectMultiple(description='Gender', index=(0, 1), options=('males', 'females', 'total'), rows=3, value=('male…

Output()

**Author and Copyright Notice** 

Amaan Khan. *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*